# Standardize survey data
**Note:** This code results in `../output/survey.sqlite` which should be moved to `$BIODIVERSITY_DATA/survey/` for dependent modules to work.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pandas import ExcelFile

SURVEY_DATA_COLUMNS=['species','latitude','longitude','phase','presence','magnitude']

## Input and output

In [2]:
DB='../output/survey.sqlite'
PHASE_ONE='../../data/field_survey/field_report_2019-06-11_phase1.xlsx'
PHASE_TWO='../../data/field_survey/field_report_2019-08-07_phase2.csv'

## Initiating dataframe
Mandatory column names for database (more can be added): `index,species,latitude,longitude,presence`

In [3]:
surveyData=pd.DataFrame(columns=SURVEY_DATA_COLUMNS)

The function `finalize()` below performs final cleaning up and appending to main dataframe.

In [4]:
def finalize(df,surveyData):
    # Verification
    if df['latitude'].dtype!='float':
        raise TypeError("Column 'latitude' should be float.")
    elif df['longitude'].dtype!='float':
        raise TypeError("Column 'longitude' should be float.")

    surveyData=pd.concat([surveyData,df],sort=True)
    print("After appending to main dataframe: ",surveyData.shape)
    return surveyData

## Phase 1
Processing one sheet at a time. The below function reads in a sheet and modifies columns.
Then, we process each species individually. Note that there are a number of duplicate entries.

In [5]:
def process_phase_one_sheet(sheetName,speciesName):
    df=pd.read_excel(PHASE_ONE,sheet_name=sheetName)
    df.columns=map(str.lower,df.columns)
    print("Original data shape: ",df.shape)
    
    # Retain required columns
    if speciesName=='Parthenium hysterophorus':
        df=df[['latitude','longitude','magnitude','presence_absence']]
    elif speciesName=='Eichhornia crassipes':
        df=df[['x_coordi','y_coordi','magnitude']]
        df.columns=['latitude','longitude','magnitude']
    else:
        df=df[['latitude','longitude','magnitude']]
    
    # Delete unwanted rows
    df=df.drop_duplicates()
    df=df[df.latitude!='Absent']
    df=df[df.longitude!='Absent']
    print("After dropping duplicates and extra columns: ",df.shape)
    
    # Create presence column
    df['presence']=False
    if speciesName=='Parthenium hysterophorus':
        df.loc[df['presence_absence']=='Presence','presence']=True
        df=df.drop(['presence_absence'],axis=1)
        df.loc[df['magnitude'].isna(),'magnitude']='Not recorded.'
    else:
        df=df[~df['magnitude'].isna() & ~df['latitude'].isna() & ~df['longitude'].isna()]
        df.loc[(df['magnitude']!='Absent') & (df['magnitude']!=''),'presence']=True
    
    # Summary
    print("Final number of points: ",df.shape)
    df['species']=speciesName
    df['phase']=1
    print("Number of presence points:",df[df['presence']==True].shape)
    print("Number of absence points:",df[df['presence']==False].shape)

    return df

### Lantana camara

In [6]:
df=process_phase_one_sheet('Lentena','Lantana camara')
surveyData=finalize(df,surveyData)
# df=pd.read_excel(PHASE_ONE,sheet_name='Lentena')
# df.columns=map(str.lower,df.columns)
# df['presence']=False
# df

Original data shape:  (538, 12)
After dropping duplicates and extra columns:  (514, 3)
Final number of points:  (514, 4)
Number of presence points: (183, 6)
Number of absence points: (331, 6)
After appending to main dataframe:  (514, 6)


//anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Chromolaena odorata

In [7]:
df=process_phase_one_sheet('Chromoleana','Chromolaena odorata')
surveyData=finalize(df,surveyData)

Original data shape:  (121, 12)
After dropping duplicates and extra columns:  (120, 3)
Final number of points:  (120, 4)
Number of presence points: (120, 6)
Number of absence points: (0, 6)
After appending to main dataframe:  (634, 6)


###  Ipomoea carnea

In [8]:
df=process_phase_one_sheet('Ipomea','Ipomoea carnea')
surveyData=finalize(df,surveyData)

Original data shape:  (288, 11)
After dropping duplicates and extra columns:  (287, 3)
Final number of points:  (286, 4)
Number of presence points: (133, 6)
Number of absence points: (153, 6)
After appending to main dataframe:  (920, 6)


### Mikania micrantha

In [9]:
df=process_phase_one_sheet('Mikeniea','Mikania micrantha')
surveyData=finalize(df,surveyData)

Original data shape:  (116, 12)
After dropping duplicates and extra columns:  (116, 3)
Final number of points:  (115, 4)
Number of presence points: (54, 6)
Number of absence points: (61, 6)
After appending to main dataframe:  (1035, 6)


### Ageratina adenophora

In [10]:
df=process_phase_one_sheet('Ageratina','Ageratina adenophora')
x=df['longitude'].tolist()
print("Errors in 'longitude'",[i for i in range(len(x)) if type(x[i])==str])
print(x[29],x[519])
df.loc[29,'longitude']=85.0775
df=df.drop(df.index[519])
x=df['longitude'].tolist()
print("Verifying after making corrections: ",[i for i in range(len(x)) if type(x[i])==str])
x=df['latitude'].tolist()
print("No errors in 'latitude'",[i for i in range(len(x)) if type(x[i])==str])
df=df.astype({'latitude':'float','longitude':'float'})
df=df[df['magnitude']!="Not recorded"]
print("Removed 'Not recorded' entries for now.")
print("Number of presence points:",df[df['presence']==True].shape)
print("Number of absence points:",df[df['presence']==False].shape)
surveyData=finalize(df,surveyData)

Original data shape:  (543, 11)
After dropping duplicates and extra columns:  (521, 3)
Final number of points:  (520, 4)
Number of presence points: (473, 6)
Number of absence points: (47, 6)
Errors in 'longitude' [29, 519]
85. 0775 Longitude
Verifying after making corrections:  []
No errors in 'latitude' []
Removed 'Not recorded' entries for now.
Number of presence points: (117, 6)
Number of absence points: (47, 6)
After appending to main dataframe:  (1199, 6)


### Parthenium hysterophorus

In [11]:
df=process_phase_one_sheet('Parthenium','Parthenium hysterophorus')
surveyData=finalize(df,surveyData)
# df=pd.read_excel(PHASE_ONE,sheet_name='Parthenium')
# df.columns=map(str.lower,df.columns)
# df['presence']=False
# df.loc[df['presence_absence']=='Presence','presence']=True

Original data shape:  (380, 12)
After dropping duplicates and extra columns:  (335, 4)
Final number of points:  (335, 4)
Number of presence points: (279, 6)
Number of absence points: (56, 6)
After appending to main dataframe:  (1534, 6)


### Eichhornia crassipes

In [12]:
df=process_phase_one_sheet('Eichorniea','Eichhornia crassipes')
surveyData=finalize(df,surveyData)

Original data shape:  (62, 10)
After dropping duplicates and extra columns:  (62, 3)
Final number of points:  (51, 4)
Number of presence points: (35, 6)
Number of absence points: (16, 6)
After appending to main dataframe:  (1585, 6)


## Phase 2
This is present in a single csv file.

In [13]:
df=pd.read_csv(PHASE_TWO)
df=df[['longitude','latitude','Species','Field  Validation status']]
df.columns=['longitude','latitude','species','presence']
df['presence_new']=False
df.loc[df['presence']=='Present','presence_new']=True
df=df.drop(['presence'],axis=1)
df=df.rename(columns={"presence_new":"presence"})
df['phase']=2
df.loc[df.species=='Lantana','species']='Lantana camara'
df.loc[df.species=='lantana','species']='Lantana camara'
df.loc[df.species=='Chromoleana','species']='Chromolaena odorata'
df.loc[df.species=='Ipomea','species']='Ipomoea carnea'
df.loc[df.species=='Mikania','species']='Mikania micrantha'
df.loc[df.species=='Ageratina','species']='Ageratina adenophora'
df.loc[df.species=='parthenium','species']='Parthenium hysterophorus'
df.loc[df.species=='Parthenium','species']='Parthenium hysterophorus'
dupl=df[df.species=='Parthenium/Lantana']
dupl.species='Lantana camara'
df.loc[df.species=='Parthenium/Lantana','species']='Parthenium hysterophorus'
df=pd.concat([df,dupl],sort=True)

# Absent columns
dupl=df[(df.species=='Absent') | (df.species=='absent')]
for sp in ['Lantana camara','Chromolaena odorata','Ipomoea carnea','Mikania micrantha','Ageratina adenophora','Parthenium hysterophorus']:
    duplSp=dupl.copy()
    duplSp.species=sp
    df=pd.concat([df,duplSp],sort=True)

df=df[(df.species!='Absent') & (df.species!='absent')]
df.species.unique()
print("Original data shape",df.shape)
df=df.dropna()
print("After dropping rows with NaNs", df.shape)
print("Number of presence points:",df[df['presence']==True].shape)
print("Number of absence points:",df[df['presence']==False].shape)
surveyData=finalize(df,surveyData)

Original data shape (434, 5)
After dropping rows with NaNs (431, 5)
Number of presence points: (125, 5)
Number of absence points: (306, 5)
After appending to main dataframe:  (2016, 6)


//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


## Push to database
Assumes that the database has the following table:
```
CREATE TABLE "survey" (
	"species"	TEXT,
	"latitude"	REAL,
	"longitude"	REAL,
    "phase"	INTEGER,
	"presence"	INTEGER,
	"magnitude"	TEXT,
	PRIMARY KEY("species","latitude","longitude","phase")
);
```

In [14]:
conn = sqlite3.connect(DB)
cur=conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS "survey" ( \
    "species"    TEXT, \
    "latitude"    REAL, \
    "longitude"    REAL, \
    "phase"    INTEGER, \
    "presence"    INTEGER, \
    "magnitude"    TEXT, \
    PRIMARY KEY("species","latitude","longitude","phase") \
);')
surveyData.to_sql('temporary_table',conn,if_exists='replace')
cur=conn.cursor()
cur.execute('INSERT OR REPLACE INTO survey SELECT ' + ','.join(SURVEY_DATA_COLUMNS) + ' FROM temporary_table;')
cur.execute('DROP TABLE temporary_table;')
conn.commit()
conn.close()

## Verification
### Distinct species list

In [15]:
conn = sqlite3.connect(DB)
cur=conn.cursor()
cur.execute('SELECT DISTINCT species from survey;')
cur.fetchall()

[('Ageratina adenophora',),
 ('Chromolaena odorata',),
 ('Eichhornia crassipes',),
 ('Ipomoea carnea',),
 ('Lantana camara',),
 ('Mikania micrantha',),
 ('Parthenium hysterophorus',)]

### Distinct values in 'presence' column

In [16]:
cur.execute('SELECT DISTINCT presence from survey;')
cur.fetchall()

[(1,), (0,)]

### Distinct values in 'magnitude' column

In [17]:
cur.execute('SELECT DISTINCT magnitude from survey;')
cur.fetchall()

[('Low',),
 ('Moderate',),
 ('Dense',),
 ('moderate',),
 ('Absent',),
 ('just invaded',),
 ('low',),
 ('dense',),
 ('High',),
 ('Very high',),
 ('Not recorded.',),
 ('absent',),
 (None,)]

### Rows in dataframe and database

In [18]:
cur.execute('SELECT count(*) FROM survey;')
numRows=cur.fetchall()
print("Number of rows in database: %d" %numRows[0])
print("Number of rows in dataframe: %d" %surveyData.shape[0])

Number of rows in database: 1976
Number of rows in dataframe: 2016


**Note:** The discrepancy in the number of rows in database and dataframe is because there are rows for which 'magnitude' differs.

### Checking presence/absence points for each species

In [19]:
cur.execute('SELECT species,phase,count(*),sum(presence) AS presence,sum((1-presence)) AS absence FROM survey GROUP BY species,phase;')
res=cur.fetchall()
for ele in res:
    print(ele)

('Ageratina adenophora', 1, 163, 117, 46)
('Ageratina adenophora', 2, 41, 0, 41)
('Chromolaena odorata', 1, 120, 120, 0)
('Chromolaena odorata', 2, 75, 12, 63)
('Eichhornia crassipes', 1, 51, 35, 16)
('Ipomoea carnea', 1, 286, 133, 153)
('Ipomoea carnea', 2, 44, 0, 44)
('Lantana camara', 1, 508, 177, 331)
('Lantana camara', 2, 147, 106, 41)
('Mikania micrantha', 1, 115, 54, 61)
('Mikania micrantha', 2, 41, 0, 41)
('Parthenium hysterophorus', 1, 302, 246, 56)
('Parthenium hysterophorus', 2, 83, 7, 76)


In [20]:
conn.close()